In [1]:
"""
Нейронная часть речевой маркировки 

Теперь мы будем решать проблему пометки POS с помощью нейронных сетей.
С точки зрения глубокого обучения, это задача прогнозирования последовательности выходных данных, 
согласованной с последовательностью входных данных. 
Есть несколько проблем, которые соответствуют этой формулировке:

- Пометка части речи - вспомогательная задача для многих проблем НЛП
- Распознавание именованных сущностей - для чат-ботов и веб-сканеров
- Прогнозирование структуры белка - для биоинформатики

Удалите CWD из sys.path, пока мы загружаем материалы.
"""

'\nНейронная часть речевой маркировки \n\nТеперь мы будем решать проблему пометки POS с помощью нейронных сетей.\nС точки зрения глубокого обучения, это задача прогнозирования последовательности выходных данных, \nсогласованной с последовательностью входных данных. \nЕсть несколько проблем, которые соответствуют этой формулировке:\n\n- Пометка части речи - вспомогательная задача для многих проблем НЛП\n- Распознавание именованных сущностей - для чат-ботов и веб-сканеров\n- Прогнозирование структуры белка - для биоинформатики\n\nУдалите CWD из sys.path, пока мы загружаем материалы.\n'

In [2]:
import nltk
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter   # Подсчет количества повторений элементов в последовательности
from collections import defaultdict # defaultdict автоматически назначает ноль как значение любому ключу, который еще не имеет значения.
import keras
import keras.layers as L
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow_addons.layers import CRF


# Процесс классификации слов по частям речи и их соответствующей маркировки известен как тегирование части речи , 
# POS-тегирование или просто тегирование . Части речи также известны как классы слов или лексические категории . 
# Набор тегов, используемых для конкретной задачи, называется набором тегов
# Тегер части речи или POS-tagger обрабатывает последовательность слов и присоединяет часть тега речи к каждому слову

# По соглашению в NLTK маркированный токен представляется с помощью кортежа, состоящего из токена и тега. 
# Мы можем создать один из этих специальных кортежей из 
# стандартного строкового представления помеченного токена, используя функцию str2tuple ()

# Некоторые из корпусов, включенных в NLTK, были помечены как часть речи. Например, Brown Corpus
# теги части речи были преобразованы в верхний регистр, 
# поскольку это стало стандартной практикой для Brown Corpus
# Когда корпус содержит текст с тегами, интерфейс корпуса NLTK будет иметь метод tagged_words ()

# Метод tagged_sents () делит помеченные слова на предложения, 
# а не представляет их как один большой список. Это будет полезно, 
# когда мы перейдем к разработке автоматических тегеров, 
# поскольку они обучаются и тестируются на списках предложений, а не слов
# Взято отсюда: категоризация и теги слов - https://www.nltk.org/book/ch05.html

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal') # универсальный набор тегов, также могут быть категории, пр, новостей
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])
data[0] # первый из списков (первое предложение)
# data
# dtype==data
# Предупреждение:
# Создание ndarray из неровных вложенных последовательностей 
# (которые представляют собой список или кортеж списков или кортежей или ndarray разной длины или формы) 
# не рекомендуется. 
# Если вы намеревались это сделать, вы должны указать "dtype=объект" при создании ndarray

# Работа с корпусами: https://www.nltk.org/api/nltk.corpus.html


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\zlatt\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\zlatt\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
C:\Users\zlatt\anaconda3\lib\site-packages\ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


[('the', 'DET'),
 ('fulton', 'NOUN'),
 ('county', 'NOUN'),
 ('grand', 'ADJ'),
 ('jury', 'NOUN'),
 ('said', 'VERB'),
 ('friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

In [3]:
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [4]:
# Чтобы добавить изображения в нашу программу, можно использовать библиотеку IPython. 
# Можно генерировать HTML напрямую, используя все его возможности
# документация https://ipython.org/ipython-doc/3/api/generated/IPython.display.html
from IPython.display import HTML, display   # display - объект, в котором хранятся данные для отображения
def draw(sentence):
    words,tags = zip(*sentence)                                          # zip() в Python определяется как zip(* iterables). 
#                                                                         Функция принимает итераторы (то есть последователь-
#                                                                         ности) в качестве аргументов 
#                                                                         и возвращает то же итератор. Этот итератор генерирует 
#                                                                         серию кортежей, содержащих элементы из каждой итерации.
#                                                                         zip() может принимать любые типы итераций, такие как 
#                                                                         файлы, списки, кортежи, словари, наборы и т. д.
#                                                                         numbers = [1, 2, 3]
#                                                                         numbers = [1, 2, 3]
#                                                                         letters = ['a', 'b', 'c']
#                                                                         zipped = zip(numbers, letters)
#                                                                         list(zipped)
#                                                                           # [(1, 'a'), (2, 'b'), (3, 'c')]
#                                                                          
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
        words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))
    
    
draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


In [5]:
# Создание словарей
# Как и раньше, мы должны построить сопоставление токенов с целочисленными идентификаторами. 
# На этот раз наша модель работает на уровне слов, обрабатывая одно слово за шаг RNN (рекурентной сети). 
# Это означает, что нам придется иметь дело с гораздо большим словарным запасом.

# К счастью для нас, мы получаем эти слова только в качестве входных данных, 
# т. е. нам не нужно их предсказывать. Это означает, что мы можем иметь 
# большой словарный запас бесплатно, используя встраивания слов.


word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0]) # most_common([n])- "самый распространенный"
                                                                                    # возвращает список из n наиболее 
                                                                                    # распространенных элементов и их кол-во 
                                                                                    # от наиболее распространенных до наименее. 
                                                                                     # Если n опущено или None, 
                                                                                    # метод cnt.most_common() 
                                                                                    # возвращает все элементы в счетчике 
               # Подробнее: https://docs-python.ru/standart-library/modul-collections-python/klass-counter-modulja-collections/
# Измерим, какая доля слов данных содержится в словаре
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [6]:
# defaultdict автоматически назначает ноль как значение любому ключу, который еще не имеет значения.
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

In [7]:
tk = list(word_to_id.items())
tk[:10]

[('#EOS#', 0),
 ('#UNK#', 1),
 ('the', 2),
 (',', 3),
 ('.', 4),
 ('of', 5),
 ('and', 6),
 ('to', 7),
 ('a', 8),
 ('in', 9)]

In [8]:
tr = list(tag_to_id.items())
tr[:10]

[('#EOS#', 0),
 ('#UNK#', 1),
 ('ADV', 2),
 ('NOUN', 3),
 ('ADP', 4),
 ('PRON', 5),
 ('DET', 6),
 ('.', 7),
 ('PRT', 8),
 ('VERB', 9)]

In [9]:
# преобразуем слова и теги в матрицу фиксированного размера
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Преобразует список имен в rnn-усваиваемую матрицу с дополнениями, добавленными после конца (end)"""
#     max(map(len,lines)) вернет длину максимально длинной строки
    max_len = max_len or max(map(len,lines))                      # позволяет обрабатывать и преобразовывать все элементы в 
                                                                  # итерируемом объекте без использования явного цикла for, 
                                                                  # метода, широко известного как сопоставление (mapping). 
                                                                  # map() полезен, 
                                                                  # когда нужно применить функцию преобразования 
                                                                  # к каждому элементу в коллекции или в массиве 
                                                                  # и преобразовать их в новый массив
                                                                    # def square(number):
                                                                    #   return number ** 2
                                                                    # numbers = [1, 2, 3, 4, 5]
                                                                    # squared = map(square, numbers)
                                                                    # или squared = map(lambda num: num ** 2, numbers)
                                                                    # list(squared)
                                                                    #    [1, 4, 9, 16, 25]
#   empty() возвращает новый массив заданной формы и типа без инициированных записей:
#   здесь мы фиксируем размер будущей матрицы через len(lines) и max_len
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        # Определяет поведение при доступе к элементу, 
        # используя синтаксис self[key]. 
        # То же относится и к протоколу изменяемых и к протоколу неизменяемых контейнеров
        # про магические методы https://habr.com/ru/post/186608/ 
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix

In [10]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

In [11]:
# Построить модель
# В отличие от нашей предыдущей работы, 
# на этот раз мы сосредоточимся на высокоуровневом интерфейсе keras для рекуррентных нейронных сетей. 
# Это так просто, как это возможно при помощи RNN, 
# все это несколько ограничивает для сложных задач, таких как seq2seq.

# По умолчанию все RNN keras применяются ко всей последовательности входных данных 
# и создают последовательность скрытых состояний 
# (return_sequences=True) или только последнее скрытое состояние 
#  (return_sequences=False). Все повторения происходят под капотом.

# В верхней части нашей модели нам нужно наносить 
#  Dense layer (плотный слой) на каждый временной шаг независимо. 
#  На данный момент по умолчанию keras.layers.Dense будет применяться один раз ко всем объединенным временным шагам. 
#  Для модификации Dense layer мы используем keras.layers.TimeDistributed,
#  чтобы он применялся как по пакетной (batch), так и по временной осям.

In [12]:
model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
# Embedding - кодирование слов
# модель принимает на вход матрицу целых чисел размерностью len(all_words) строк на 50 столбцов 
model.add(L.Embedding(len(all_words),50))    
# SimpleRNN() - полносвязная RNN в которой выход предыдущего временного шага 
# должен быть передан в следующий шаг
# Слой RNN может также возвращать всю 
# последовательность выходных данных для каждого элемента 
# (по одному вектору на каждый шаг), если указываем return_sequences=True
model.add(L.SimpleRNN(64,return_sequences=True))

#добавим верхний слой, который предсказывает вероятности тегов
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
# L.TimeDistributed() - эта оболочка позволяет применять слой к каждому временному фрагменту ввода
# Каждый вход должен быть как минимум трехмерным, 
# и размер первого индекса первого входа будет считаться временным измерением.
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [13]:
# Обучение: в этом случае мы не хотим заранее 
# готовить весь набор данных для обучения. 
# Основная причина заключается в том, 
# что длина каждого пакета зависит 
# от максимальной длины предложения в пакете. 
# Это оставляет нам два варианта: 
# использовать пользовательский обучающий код, 
# как на предыдущем семинаре, или использовать генераторы.

# В моделях Keras есть метод model.fit_generator, 
# который принимает генератор python, 
# выдающий по одной партии за раз. 
# Но сначала нам нужно реализовать такой генератор

In [14]:
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),'Убедитесь, что предложения - это массив numpy array'
    
    while True:
        # random.permutation() возвращает случайную перестановку (permutation)
        # элементов массива или случайную последовательность заданной длинны из его элементов.
        # Данная функция выполняет перестановку только по первой оси, 
        # поэтому для многомерных массивов возвращается перестановка его подмассивов,
        # в то время как содержание этих подмассивов не перемешивается
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot
            # Yield — это ключевое слово в Python, которое используется для возврата из функции 
            # с сохранением состояния ее локальных переменных, 
            # и при повторном вызове такой функции выполнение продолжается с оператора yield, 
            # на котором ее работа была прервана

In [15]:
# Обратные вызовы: Еще одна вещь, которая нам нужна, - 
# это измерение производительности модели. 
# Сложность заключается в том, чтобы не подсчитывать точность 
# после окончания предложения (при заполнении) и убедиться, 
# что мы подсчитываем все данные проверки ровно один раз.

# Хотя нет ничего невозможного в том, 
# чтобы убедить Keras сделать все это, 
# мы можем также написать наш собственный колбэк, 
# который сделает это. Колбэки Keras 
# позволяют нам писать пользовательский код, 
# который будет выполняться один раз в каждую эпоху 
# или каждый мини-batch. Мы определим его с помощью LambdaCallback

In [16]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #предскажем вероятности тегов размера [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    # argmax() возвращает индекс максимального значения вдоль указанной оси
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #вычислим точность без учета заполнения
    # (np.logical_and) вычисляет значение истинности x1 и x2 поэлементно.
    # x = np.arange(5)
    # np.logical_and(x>1, x<4)
       # array([False, False,  True,  True, False])
    # в numerator должны получить пересечение истинности по двум позициям:
    # предсказанные теги равны тестовым и и тестовые слова этих тегов не равны нулю
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    """Оценка точности"""
    def on_epoch_end(self,epoch,logs=None):
        # Стандартный вывод Python является буферизированным (это означает, 
        # что он собирает некоторые данные, «записанные» в стандарте, 
        # прежде чем записать их в терминал). Вызов sys.stdout.flush()заставляет его «очистить» буфер, 
        # что означает, что он будет записывать все, 
        # что есть в буфере, на терминал, даже если обычно он будет ждать перед этим.
        # подробнее о буферах данных - https://en.wikipedia.org/wiki/Data_buffer 
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()
        

In [17]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

C:\Users\zlatt\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1343/1343 [==============================] - 19s 14ms/step - loss: 0.2452

Measuring validation accuracy...
448/448 [==============================] - 5s 10ms/step

Validation accuracy: 0.94050

Epoch 2/5
1343/1343 [==============================] - 18s 13ms/step - loss: 0.0577

Measuring validation accuracy...
448/448 [==============================] - 4s 9ms/step

Validation accuracy: 0.94522

Epoch 3/5
1343/1343 [==============================] - 19s 14ms/step - loss: 0.0505

Measuring validation accuracy...
448/448 [==============================] - 5s 11ms/step

Validation accuracy: 0.94617

Epoch 4/5
1343/1343 [==============================] - 22s 16ms/step - loss: 0.0460

Measuring validation accuracy...
448/448 [==============================] - 5s 11ms/step

Validation accuracy: 0.94640

Epoch 5/5
1343/1343 [==============================] - 22s 16ms/step - loss: 0.0418

Measuring validation accuracy...
448/448 [==============================] - 5s 11ms/step

Valida

In [18]:
# Измерим конечную точность на всем наборе тестов
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras снова взбесился, пожалуйста, свяжитесь с персоналом курса."

448/448 [==============================] - 5s 11ms/step
Final accuracy: 0.94553


In [19]:
# Движение в двух направлениях
# Поскольку мы анализируем полную последовательность, для нас законно изучать будущие данные.

# Простой способ добиться этого-идти в обоих направлениях одновременно, 
# создавая двунаправленный RNN (bidirectional RNN).
# В Keras мы можем добиться этого как вручную (с помощью двух LSTM и объединения), так и с помощью keras.layers.Bidirectional.

# Этот работает так же, как и распределенный по времени, который мы видели ранее: 
# вы обертываете его вокруг повторяющегося слоя (сейчас SimpleRNN, а позже LSTM/GRU), 
# и он фактически создает два слоя под капотом.
# Ваша первая задача-использовать такой слой нашего POS-теггера.

In [20]:
#Определим модель, которая использует bidirectional SimpleRNN
# from keras.layers import SimpleRNN, Bidirectional, Embedding, LSTM, Dense

model = keras.models.Sequential()

# <Your code here!>


model.add(L.Embedding(len(all_words),50))

forward_layer = L.SimpleRNN(64, return_sequences=True) # слой можно заменить на LSTM (более продвинут)
backward_layer = L.SimpleRNN(64, activation='relu', return_sequences=True, go_backwards=True)
model.add(L.Bidirectional(forward_layer, backward_layer=backward_layer))

stepwise_dense = L.Dense(len(all_tags),activation='softmax') # для полносвязной сети Dense здесь обязательна с функцией softmax
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)
model.summary()
model.compile('adam','categorical_crossentropy')
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          500100    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         14720     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 14)          1806      
Total params: 516,626
Trainable params: 516,626
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1343/1343 [==============================] - 37s 26ms/step - loss: 0.2091

Measuring validation accuracy...
448/448 [==============================] - 9s 20ms/step

Validation accuracy: 0.95555

Epoch 2/5
1343/1343 [==============================] - 36s 26ms/step - loss: 0.0441

Measuring validation accuracy...
448/448 [==============================] - 9s 19ms/step

In [21]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 8s 17ms/step

Final accuracy: 0.96089
Well done!


In [22]:
# Задача I: Структурированные функции потерь (больше бонусных баллов)

# Поскольку мы помечаем всю последовательность сразу, мы могли бы также обучить нашу сеть этому. 
# Помните линейный CRF из лекции? Вы также можете использовать его 
# в качестве функции потерь для вашего RNN

# Существует несколько способов сделать это, 
# но мы бы рекомендовали начать с условных случайных полей
# Вы можете подключить CRF в качестве функции потерь и по-прежнему тренироваться с помощью backprop. 
# Для вас даже есть какая-то аккуратная реализация tensorflow.
# Кроме того, вы можете настроить свою модель на предыдущие теги 
# (сделать ее авторегрессионной) и выполнить поиск луча по этой модели.

In [23]:
# Несколько советов
# Здесь есть еще несколько советов о том, как улучшить обучение, 
# которые немного сложнее реализовать. Мы настоятельно рекомендуем вам 
# попробовать их после того, как у вас будет хорошая начальная модель.

# 1) Используйте предварительно подготовленные вложения (pre-trained embeddings): вы можете 
# использовать предварительно подготовленные веса оттуда, чтобы запустить свой слой встраивания.
#   -Слой встраивания имеет матрицу W (слой.W), который содержит встраивания слов 
# для каждого слова в словаре. Вы можете просто перезаписать их с помощью tf.assign.
#   - При использовании предварительно подготовленных вложений 
# обратите внимание на то, что словарь модели отличается от вашего собственного.
#   - Возможно, вам захочется переключить trainable=FALSE для встраивания слоя в первые несколько эпох, 
# как при обычной тонкой настройке.
# 2) Выходите за рамки SimpleRNN: есть keras.layers.LSTM и keras.layers.GRU
#   - Если вы хотите использовать пользовательскую рекуррентную ячейку, прочтите это
#   - Вы также можете использовать 1D свертки (keras.слои.Conv1D). 
# Они часто так же хороши, как и повторяющиеся слои, но с меньшей подгонкой.
# 3) Укладывайте больше слоев: если в этом курсе есть общий мотив, то он касается укладки слоев
#   - Вы можете просто добавить повторяющиеся и 1dconv слои друг на друга, и керас поймет это
#   - Просто помните, что более крупным сетям может потребоваться больше эпох для обучения
# 4) Регуляризация: вы можете применять отсевы как обычно, но также и специфичным для RNN способом
#   - keras.layers.Dropout работает между слоями RNN
#   - Рекуррентные слои также имеют параметр recurrent_dropout
# 5) Обрезка градиента (Gradient clipping): если ваше обучение не так стабильно, как вам хотелось бы, 
# установите clipnorm в своем оптимизаторе.
#   - То есть, это хорошая идея - следить за своей кривой потерь при каждом мини-матче. П
# опробуйте tensorboard callback или что-то подобное.
# 6) Отсев слов (Word Dropout): tl; dr случайным образом заменяет слова на UNK во время обучения.
#   - Это также может имитировать увеличение количества неизвестных слов в наборе тестов
# 7) Больший словарный запас: Вы можете повысить производительность, 
# увеличив входной словарь вашей модели с 5000 до каждого отдельного слова!
#   - Просто убедитесь, что ваша модель не подходит по многим параметрам.
#   - В сочетании с регуляризаторами или предварительно обученными векторами слов 
# это может быть действительно хорошо, потому что сейчас наша модель слепа к > 5% слов.
# 8) Более эффективная дозировка (batching): прямо сейчас TF тратит много времени на повторение "0" с
#   - Это происходит потому, что пакет всегда дополняется до длины самого длинного предложения
#   - Вы можете ускорить процесс, предварительно сгенерировав пакеты одинаковой длины 
# и снабдив их случайно выбранным предварительно сгенерированным пакетом.
#   - Это технически нарушает предположение о i.i.d., но это работает, 
# если вы не придумаете какие-то безумные архитектуры rnn.
# 9) Самый важный совет: не втискивайте все сразу!
#   - Если вы добавите много модификаций, некоторые из них почти неизбежно окажутся вредными, 
# и вы никогда не узнаете, какие из них есть.
#   - Попробуйте вместо этого выполнить небольшие итерации и записать результаты эксперимента, 
# чтобы направлять дальнейший поиск.
# Удачной охоты!

In [25]:
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy

inputs = L.Input(shape=(None,))     # если делаем через Sequential, то input и output не нужны 
x = L.Embedding(len(all_words),50)(inputs)
x = L.Bidirectional(L.SimpleRNN(64, return_sequences=True))(x)
x = L.Bidirectional(L.SimpleRNN(32, activation='relu', return_sequences=True, go_backwards=True))(x)
x = L.Dense(len(all_tags),activation='softmax')(x)
crf = CRF(len(all_tags))
outputs = crf(x)
model = keras.Model(inputs, outputs)


In [26]:
model.compile('adam', loss='categorical_crossentropy')
model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,callbacks=[EvaluateAccuracy()],
                    epochs=5,)

Epoch 1/5


ValueError: in user code:

    C:\Users\zlatt\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\zlatt\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\zlatt\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\zlatt\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\zlatt\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\zlatt\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\z

In [ ]:
# keras.test.is_gpu_available()
# from keras_contrib.layers import CRF
# from keras_contrib.losses import crf_loss
# from keras_contrib.metrics import crf_viterbi_accuracy